# Convolutional Neural Network 모델 정의

In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [5]:
# Conv2d 생성
layer = nn.Conv2d(
    in_channels=3,  # 입력 데이터의 channel 개수. 입력 tensor의 shape: (batch_size, channel, height, width) 
    out_channels=5, # 필터의 개수 (output feature map의 개수)
    kernel_size=3,  # 필터의 크기 (3, 3)
    stride=1,       # 계산을 위하 이동 크기. 좌->우: 1칸씩, 상->하: 1칸 (default: 1)
    padding=1,      # 패팅 크기 (정수: 상하/좌우 동일할 패팅크기를 명시 - 0(default): 패딩추가 안함.)
                    # "same": 입력 size와 동일한 size의 출력이 나오도록 알아서 패딩을 추가.
)
# 다음 layer 생성
# layer2 = nn.Conv2d(
#     in_channels=5, # 이전 Conv2d의 out_channels 값이 다음 Conv2d의 in_channels 값이 된다.
# )

In [6]:
input_data = torch.ones(1, 3, 10, 10) # batch크기, channel수, heigth, width
output = layer(input_data)
output.shape

torch.Size([1, 5, 10, 10])

In [7]:
(10 - 3 + 2*1)/1 + 1

10.0

In [10]:
### Conv2d의 weigth 의 shape
layer.weight.shape
# [5:필터개수-out_channels, 
#  3:channel수-in_channel, 
#  3:height-kernel_size, 
#  3:width-kernel_size]

torch.Size([5, 3, 3, 3])

In [12]:
layer.bias.shape # channel당 1개씩 bias가 추가.

torch.Size([5])

In [25]:
pool_layer = nn.MaxPool2d(
    kernel_size=2, # 값을 추출하는 영역 크기(2, 2) - default: 2
    stride=2,      # 다음 값을 추출하기위해서 몇칸을 이동할지.(default: kernel_size)
    padding=0
    # 값을 추출할 영역이 kernel_size보다 작을 경우 추출할지 여부.
    # 0-추출을 하지 않겠다.
)

In [19]:
print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

torch.Size([1, 3, 10, 10])
torch.Size([1, 3, 5, 5])


In [26]:
input_data = torch.randn(1, 1, 5, 5)

print(input_data.shape)
pool_output = pool_layer(input_data)
print(pool_output.shape)

torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 2, 2])


In [27]:
input_data[0, 0]

tensor([[ 0.7670,  0.3891,  0.0278, -1.1643, -1.3077],
        [ 0.4071,  0.0203,  0.2442, -0.0384,  1.5362],
        [ 0.8467,  1.0411, -2.5247,  0.2642,  1.2496],
        [ 2.5154,  0.0489, -0.5061, -1.4856,  0.3671],
        [-0.4352,  0.5259,  0.5502,  0.6428,  0.0199]])

In [28]:
pool_output

tensor([[[[0.7670, 0.2442],
          [2.5154, 0.2642]]]])

# MNIST

In [30]:
%pip install torchinfo

  Using cached torchinfo-1.8.0-py3-none-any.whl.metadata (21 kB)
Using cached torchinfo-1.8.0-py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os

import torch
from torch import nn
import torchinfo

import matplotlib.pyplot as plt
import numpy as np

from module.data import load_mnist_dataset, load_fashion_mnist_dataset
from module.train import fit

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [33]:
# 하이퍼파라미터 지정
EPOCH = 1
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DATA_ROOT_DIR = "datasets"

## Data 준비

In [34]:
train_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, True)
test_loader = load_mnist_dataset(DATA_ROOT_DIR, BATCH_SIZE, False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.07MB/s]


Extracting datasets\MNIST\raw\train-images-idx3-ubyte.gz to datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 159kB/s]


Extracting datasets\MNIST\raw\train-labels-idx1-ubyte.gz to datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.62MB/s]


Extracting datasets\MNIST\raw\t10k-images-idx3-ubyte.gz to datasets\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.55MB/s]

Extracting datasets\MNIST\raw\t10k-labels-idx1-ubyte.gz to datasets\MNIST\raw



## CNN 모델 정의

In [ ]:
# CNN - layer block
# ConvolutionLayer -> Activation -> Pooling Layer
# ConvolutionLayer -> Activation -> ConvolutionLayer -> Activation-> Pooling Layer

# ConvolutionLayer -> BatchNormalization -> Activation -> Dropout -> Pooling Layer

## 구조: filter 개수는 늘려주고(channel-depth) feature map의 size는 줄이는 방식으로 구성.
# depth: Conv2d,  size: MaxPool2d

class CNNModel(nn.Module):

    def __init__(self, dropout_rate=0.2):
        # 모델을 구성하는 Layer함수들을 초기화(객체 생성)
        super().__init__()
        # block 단위로 정의 - nn.Sequential()
        self.b1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1, # grayscale 이미지 -> channel수: 1, color: 3
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(32), # Conv: out_channels
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        self.b2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32, # b1 채널개수(out_channels)
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(64), # Conv: out_channels
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        self.b3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64, # b2 채널개수(out_channels)
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding="same"
            ),
            nn.BatchNorm2d(128), # Conv: out_channels
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        # 추론기(분류기) - Fully Conntected Layer(nn.Liear)
        self.classifier = 




    def forward(self, X):
        # 추론(연산처리)
        pass

## Train